In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
import scipy.stats as stat
from sklearn.preprocessing import StandardScaler
pd.pandas.set_option('display.max_columns', None)
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn import preprocessing

In [ ]:
df=pd.read_csv("/kaggle/input/titanic/train.csv")

In [ ]:

df.head()

In [ ]:
df.dtypes

# Categorical Features

In [ ]:
categorical_features=[features for features in df.columns if df[features].dtypes=='object']
categorical_features

# Numerical Features

In [ ]:
numerical_features=[feature for feature in df.columns if df[feature].dtypes!='object']
numerical_features

# Creating some new variables


In [ ]:
# Extracting Titles fron names
list1=[]
k=0
for j in df.Name:
    
    list1.append(str(j).split()[1])
    k+=1
    

In [ ]:
df['Title']=list1
df.head(1)

In [ ]:
# Family Size
df['Family_size']=df['SibSp']+df['Parch']+1
df.head(1)

In [ ]:
# Age groups
list2=df.Age.tolist()
list3=['teen-age' if z<18 else 'adult' if z<60 else 'old' for z in list2]
df['Age_group']=list3
df.head(1)

# Missing values

In [ ]:
df.isnull().sum()

In [ ]:
mode=df.Embarked.value_counts().index[0]
df['Embarked'].fillna(mode,inplace=True)

In [ ]:
# for Age column we will use random sample imputer
def impute_age(df,variable):
    df[variable+'random']=df[variable]
    random_sample=df[variable].dropna().sample(df[variable].isnull().sum(),random_state=0)
    random_sample.index=df[df[variable].isnull()].index
    df.loc[df[variable].isnull(),variable+'random']=random_sample

In [ ]:
impute_age(df,'Age')

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
df['Agerandom'].plot(kind='kde', ax=ax)
df.Age.plot(kind='kde', ax=ax, color='green')
lines, labels = ax.get_legend_handles_labels()
ax.legend(lines, labels, loc='best')


#### this figure shows that imputing missing values with random samples has not changed the distribution of data


In [ ]:
# extracting the cabin names
list4=[]
df['Cabin']=df["Cabin"].astype(str)
for z in df['Cabin']:
    list4.append(str(z.split()[0][0]))
    

In [ ]:
df['Cabin_new']=list4

In [ ]:
df.Cabin_new.value_counts()

In [ ]:
# since m indicates missing values we will replace it by missing
df['Cabin_new'].replace('n','missing',inplace=True)

# Visualization

In [ ]:
df.Survived.value_counts().plot(kind='bar')
print(df.Survived.value_counts())

In [ ]:
df.Sex.value_counts().plot(kind='bar')
print(df.Sex.value_counts())

In [ ]:
df.Pclass.value_counts().plot(kind='bar')
print(df.Pclass.value_counts())

In [ ]:
df.Cabin_new.value_counts().plot(kind='bar')
print(df.Cabin_new.value_counts())

In [ ]:
df.Embarked.value_counts().plot(kind='bar')
print(df.Embarked.value_counts())

In [ ]:
df.Title.value_counts().plot(kind='bar')
print(df.Title.value_counts().head(10))

In [ ]:
df.Age_group.value_counts().plot(kind='bar')
print(df.Age_group.value_counts())

In [ ]:
sns.distplot(df.Agerandom)

In [ ]:
sns.distplot(df.Fare)

In [ ]:
df.Parch.value_counts().plot(kind='bar')
print(df.Parch.value_counts())

In [ ]:
df.SibSp.value_counts().plot(kind='bar')
print(df.SibSp.value_counts())

In [ ]:
df.Age_group.value_counts().plot(kind='bar')
print(df.Age_group.value_counts())

# Bivariate Analysis

In [ ]:
sns.catplot(x = 'Sex', hue = 'Survived',data = df, kind = 'count')

In [ ]:
sns.catplot(x = 'Cabin_new', hue = 'Survived',data = df, kind = 'count')

In [ ]:
sns.catplot(x = 'Age_group', hue = 'Survived',data = df, kind = 'count')

In [ ]:
sns.catplot(x = 'Embarked', hue = 'Survived',data = df, kind = 'count')

# Encoding

In [ ]:
# Pclass
label_encoder = preprocessing.LabelEncoder() 
df['Pclass']=label_encoder.fit_transform(df['Pclass'])

In [ ]:
# Sex
sex=pd.get_dummies(df['Sex'],drop_first=True)

In [ ]:
# Embarked
Embarked=pd.get_dummies(df['Embarked'],drop_first=True)

In [ ]:
# titles
### since we have a lot of titles so we will take top 4 and mark rest as 0
most_frequent_titles=df.Title.value_counts().sort_values(ascending=False).head(4).index
most_frequent_titles=list(most_frequent_titles)
most_frequent_titles

In [ ]:
for titles in most_frequent_titles:
    df[titles]=np.where(df['Title']==titles,1,0)

In [ ]:
# Age group
age_group=pd.get_dummies(df['Age_group'],drop_first=True)

In [ ]:
#cabin
cabin=pd.get_dummies(df['Cabin_new'],drop_first=True)


In [ ]:
df.head(1)

In [ ]:
df.drop(['PassengerId','Name','Sex','Ticket','Embarked','Cabin','Title','Age_group','Cabin_new','Age'],axis=1,inplace=True)

In [ ]:
df=pd.concat([df,age_group,cabin,Embarked,sex],axis=1)

In [ ]:
df.head()

In [ ]:
df.shape

# Hyper parameter optimizaton

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1500, num = 15)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state = 42)

In [ ]:
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 5, verbose=2, random_state=42, n_jobs = -1)

In [ ]:
X=df.iloc[:,1:]
y=df.iloc[:,0]
X.shape
y.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
print('the shape of x_train is {}'.format(X_train.shape))
print('the shape of x_test is {}'.format(X_test.shape))
print('the shape of y_train is {}'.format(y_train.shape))
print('the shape of y_test is {}'.format(y_test.shape))

In [ ]:
rf_random.fit(X_train,y_train)

In [ ]:
rf_random.best_params_

In [ ]:
rf_random.best_score_

In [ ]:
pred=rf_random.predict(X_test)

In [ ]:
confusion_matrix(y_test,pred)

In [ ]:
accuracy_score(y_test,pred)